In [8]:
import sqlite3
import pandas as pd
from datetime import datetime
import json, html, unicodedata, re

DB_PATH = "boxing_odds_staging.db"
OUTPUT = "best_odds_dashboard.html"
POSSIBLE_DATE = "2025-12-31"  # "possible matchup" placeholder date

# ---------- helpers ----------
def py_normalize(s: str) -> str:
    if s is None:
        return ""
    # remove accents/diacritics, collapse spaces, lowercase
    s = unicodedata.normalize("NFD", str(s))
    s = "".join(ch for ch in s if unicodedata.category(ch) != "Mn")
    s = re.sub(r"\s+", " ", s).strip().lower()
    return s

def escape_js_str(s: str) -> str:
    return s.replace("\\", "\\\\").replace("'", "\\'")

# ---------- load data ----------
conn = sqlite3.connect(DB_PATH)

latest = pd.read_sql("""
SELECT *
FROM odds_history
WHERE (event_id, fighter, bookmaker, observed_at) IN (
    SELECT event_id, fighter, bookmaker, MAX(observed_at)
    FROM odds_history
    GROUP BY event_id, fighter, bookmaker
)
""", conn)

history = pd.read_sql("""
SELECT event_id, fighter, observed_at, decimal_odds
FROM odds_history
ORDER BY observed_at
""", conn)
conn.close()

# ---------- prep ----------
latest["commence_time"] = pd.to_datetime(latest["commence_time"], errors="coerce")
latest["Fight Date"] = latest["commence_time"].dt.strftime("%Y-%m-%d")
latest["Matchup"] = latest["home_team"].fillna("") + " vs " + latest["away_team"].fillna("")
latest["fighter"] = latest["fighter"].astype(str).str.strip()
history["fighter"] = history["fighter"].astype(str).str.strip()
history["observed_at"] = pd.to_datetime(history["observed_at"], errors="coerce")

# remove draws
latest = latest[latest["fighter"].str.lower() != "draw"].copy()

# best odds per fighter
idx = latest.groupby(["event_id", "fighter"])["decimal_odds"].idxmax()
best = latest.loc[idx, ["event_id", "Fight Date", "Matchup", "fighter", "bookmaker", "decimal_odds"]].copy()
best["Implied %"] = (100.0 / best["decimal_odds"]).round(2)
best = best.sort_values(["Fight Date", "Matchup", "fighter"])

# split scheduled vs possible
scheduled_best = best[best["Fight Date"] != POSSIBLE_DATE].copy()
possible_best  = best[best["Fight Date"] == POSSIBLE_DATE].copy()

# ---------- build history + indexes (normalize on PY side) ----------
# HISTORY: real keys "event||fighter" -> series
HISTORY = {}
# HISTORY_INDEX: normalized "event||fighter" -> real key
HISTORY_INDEX = {}

for (ev, f), g in history.groupby(["event_id", "fighter"]):
    real_key = f"{ev}||{f}"
    norm_key = f"{py_normalize(ev)}||{py_normalize(f)}"
    HISTORY_INDEX[norm_key] = real_key
    HISTORY[real_key] = {
        "dates": g["observed_at"].dt.strftime("%Y-%m-%d %H:%M:%S").tolist(),
        "odds":  g["decimal_odds"].round(3).tolist(),
        "fighter": f
    }

# ---------- table rendering ----------
def row_html(r, label_possible=False):
    real_key = f'{r["event_id"]}||{r["fighter"]}'
    norm_key = f'{py_normalize(r["event_id"])}||{py_normalize(r["fighter"])}'
    # link will pass the *normalized* key; JS will map via HISTORY_INDEX
    safe_norm = escape_js_str(norm_key)
    badge = """<span class="badge badge-possible">Possible matchup</span>""" if label_possible else ""
    return f"""
    <tr>
      <td>{html.escape(r["Fight Date"])}{badge}</td>
      <td>{html.escape(r["Matchup"])}</td>
      <td><a class="show-chart" href="#" onclick="return showChart('{safe_norm}');">{html.escape(r["fighter"])}</a></td>
      <td>{html.escape(r["bookmaker"])}</td>
      <td><b>{round(float(r["decimal_odds"]),3)}</b></td>
      <td>{r["Implied %"]}%</td>
    </tr>
    """

def table_html(df, label_possible=False):
    rows = "\n".join(row_html(r, label_possible) for _, r in df.iterrows())
    return f"""
    <table>
      <thead>
        <tr>
          <th>Fight Date</th>
          <th>Matchup</th>
          <th>Fighter</th>
          <th>Best Bookmaker</th>
          <th>Best Odds</th>
          <th>Implied %</th>
        </tr>
      </thead>
      <tbody>
        {rows}
      </tbody>
    </table>
    """

if not scheduled_best.empty:
    parts = []
    for d, grp in scheduled_best.groupby("Fight Date"):
        parts.append(f"<div class='card'><div class='card-title'>Fights on {d}</div>{table_html(grp)}</div>")
    scheduled_section = "\n".join(parts)
else:
    scheduled_section = "<div class='card'><div class='empty'>No scheduled fights found.</div></div>"

possible_section = ""
if not possible_best.empty:
    possible_section = f"<div class='card'><div class='card-title'>Possible Matchups (TBD)</div>{table_html(possible_best, label_possible=True)}</div>"

# ---------- build HTML ----------
html_out = f"""
<!DOCTYPE html>
<html>
<head>
<meta charset="UTF-8">
<title>Best Boxing Odds — {datetime.now().strftime("%Y-%m-%d")}</title>
<script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
<style>
  body {{
    font-family: 'Segoe UI', Tahoma, sans-serif;
    background: #121212; color: #fff; margin:0; padding: 24px;
  }}
  h1 {{ text-align:center; margin: 0 0 8px; color: #ffcc00; }}
  .sub {{ text-align:center; color: #bbb; margin-bottom: 20px; }}
  .grid {{ display: grid; gap: 16px; grid-template-columns: 1fr; max-width: 1200px; margin: 0 auto; }}
  .card {{
    background: #1a1a1a; border: 1px solid #2b2b2b; border-radius: 12px;
    box-shadow: 0 10px 30px rgba(0,0,0,.25); padding: 16px; width: 100%;
  }}
  .card-title {{ font-weight:700; margin-bottom:12px; }}
  table {{ width: 100%; border-collapse: collapse; overflow: hidden; border-radius: 10px; }}
  th, td {{ padding: 12px 14px; text-align: left; border-bottom: 1px solid #2b2b2b; }}
  th {{
    background: #ffcc00; color: #000; text-transform: uppercase; font-size: 12px; letter-spacing: .5px;
    position: sticky; top:0; z-index:1;
  }}
  tr:hover {{ background: #242424; }}
  td b {{ color: #00ff99; }}
  a.show-chart {{ color: #00bfff; text-decoration: none; cursor: pointer; }}
  a.show-chart:hover {{ text-decoration: underline; }}
  #chart-card {{ display:none; max-width: 1200px; margin: 16px auto; }}
  #chart {{ width:100%; height:420px; background:#101010; border:1px solid #2b2b2b; border-radius: 10px; }}
  .empty {{ color:#aaa; }}
  .badge {{ display:inline-block; margin-left:8px; padding:2px 8px; border-radius:999px; font-size:11px; font-weight:700; vertical-align:middle; }}
  .badge-possible {{ background:#2b2b2b; color:#ffd166; border:1px solid #3a3a3a; }}
  .debug {{ color:#aaa; font-size:12px; margin-top:10px; }}
</style>
</head>
<body>
  <h1>Best Boxing Odds</h1>
  <div class="sub">Click a fighter to see their odds movement. Fights dated {POSSIBLE_DATE} are <b>possible matchups</b> (not confirmed).</div>

  <div class="grid">
    {scheduled_section}
    {possible_section}
  </div>

  <div id="chart-card" class="card">
    <div id="chart-title" style="margin:0 0 12px; font-weight:700;"></div>
    <div id="chart"></div>
    <div id="debug" class="debug"></div>
  </div>

<script>
const HISTORY = {json.dumps(HISTORY)};
const HISTORY_INDEX = {json.dumps(HISTORY_INDEX)};

function showChart(normKey) {{
  console.log("showChart norm key:", normKey);
  const debug = document.getElementById('debug');
  const chartCard = document.getElementById('chart-card');
  const title = document.getElementById('chart-title');
  const chartDiv = document.getElementById('chart');

  const realKey = HISTORY_INDEX[normKey];
  if (!realKey) {{
    title.textContent = "No odds history available";
    chartDiv.innerHTML = "";
    chartCard.style.display = 'block';
    debug.textContent = "Key not found in HISTORY_INDEX: " + normKey + " | keys: " + Object.keys(HISTORY_INDEX).length;
    return false;
  }}

  const payload = HISTORY[realKey];
  if (!payload || !payload.dates || payload.dates.length === 0) {{
    title.textContent = "No odds history available";
    chartDiv.innerHTML = "";
    chartCard.style.display = 'block';
    debug.textContent = "No payload for resolved key: " + realKey;
    return false;
  }}

  title.textContent = "Odds Movement — " + payload.fighter;
  debug.textContent = "Resolved key: " + realKey + " | points: " + payload.dates.length;

  const trace = {{
    x: payload.dates,
    y: payload.odds,
    mode: 'lines+markers',
    type: 'scatter',
    name: payload.fighter,
    line: {{ width: 2 }}
  }};
  const layout = {{
    paper_bgcolor: '#101010',
    plot_bgcolor: '#101010',
    font: {{ color: '#eaeaea' }},
    margin: {{ t: 20, r: 16, b: 40, l: 55 }},
    xaxis: {{ title: 'Time', gridcolor: '#2b2b2b' }},
    yaxis: {{ title: 'Decimal Odds', gridcolor: '#2b2b2b' }}
  }};
  Plotly.newPlot('chart', [trace], layout, {{ displayModeBar: false, responsive: true }});
  chartCard.style.display = 'block';
  return false;
}}
window.showChart = showChart;
</script>
</body>
</html>
"""

with open(OUTPUT, "w", encoding="utf-8") as f:
    f.write(html_out)

print(f"✅ Wrote {OUTPUT} — open it directly in your browser or via a local server (python -m http.server)")


✅ Wrote best_odds_dashboard.html — open it directly in your browser or via a local server (python -m http.server)
